# Boltzmann Machine

## Downloading the dataset

## Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn.parallel
import torch.utils.data

## Importing the dataset


In [2]:
# We won't be using this dataset.
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

<h3>Movies</h3>
<br>Columns: movie id, movie title, movie genre

In [3]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


<h3>Users</h3>
<br>Columns: user id, gender, age, job, idk

In [4]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


<h3>Ratings</h3>
<br>Columns: user id, movie id, rating, timestamp

In [5]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## Preparing the training set and the test set


In [6]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [7]:
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [8]:
test_set

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

## Getting the number of users and movies


In [9]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

In [10]:
print(f"Number of users: {nb_users}")
print(f"Number of movies: {nb_movies}")

Number of users: 943
Number of movies: 1682


## Converting the data into an array with users in lines and movies in columns


In [11]:
# Assisted by WCA@IBM
# Latest GenAI contribution: ibm/granite-20b-code-instruct-v2
def convert(data: np.ndarray) -> list:
  """
  Converts the input data into a list of lists format, where each inner list represents the ratings of a single user.

  Parameters:
      data (np.ndarray): The input data as a numpy array of shape (num_ratings, 3).

  Returns:
      list: The converted data in the form of a list of lists.
  """
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data

training_set = convert(training_set)
test_set = convert(test_set)

In [12]:
print(f"Number of lists: {len(training_set)}")
print(f"Number of ratings for a user: {len(training_set[0])}")

Number of lists: 943
Number of ratings for a user: 1682


## Converting the data into Torch tensors


In [13]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [14]:
print(training_set)

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])


In [15]:
print(f"Number of lists: {len(training_set)}")
print(f"Number of ratings for a user: {len(training_set[0])}")

Number of lists: 943
Number of ratings for a user: 1682


## Converting the ratings into binary ratings 

1 (Liked) or 0 (Not Liked)


In [16]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [17]:
print(training_set)

tensor([[-1.,  1.,  1.,  ..., -1., -1., -1.],
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1.,  1., -1.,  ..., -1., -1., -1.]])


## Creating the architecture of the Neural Network


In [18]:
class RBM():
  
  def __init__(self, nv, nh):
    """Create initial variables
    W - weights (hidden nodes, visible nodes)
    a - bias for probability of hidden nodes given visible nodes (see Gibbs sampling)
    b - bias for probability of visible nodes given hidden nodes (see Gibbs sampling)

    Args:
        nv (_type_): Visible nodes
        nh (_type_): Hidden nodes
    """
    self.W = torch.randn(nh, nv)
    self.a = torch.randn(1, nh)
    self.b = torch.randn(1, nv)
    
  def sample_h(self, x):
    """Sampling the hidden nodes according to the prob 
    of p(h) given v (h hidden nodes, v is visible nodes)
    using Gibbs sampling

    Args:
        x (_type_): Visible nodes (v) in the prob p(h) given v

    Returns:
        _type_: probability, sample of the nodes based on the prob
    """
    #product of x and W
    wx = torch.mm(x, self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)
  
  def sample_v(self, y):
    """Sampling the visible nodes according to the prob 
    of p(v) given h (h hidden nodes, v is visible nodes)
    using Gibbs sampling

    Args:
        y (_type_): Hidden nodes (h) in the prob p(v) given h

    Returns:
        _type_: probability, sample of the nodes based on the prob
    """
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)
  
  def train(self, v0, vk, ph0, phk):
    """Training by Contrastive Divergence
    see: https://www.geeksforgeeks.org/contrastive-divergence-in-restricted-boltzmann-machines/
    
    Args:
        v0 (_type_): input vector (ratings from the user for all movies)
        vk (_type_): visible nodes after k samplings
        ph0 (_type_): prob of first iteration hidden nodes equal 1 given the values of v0
        phk (_type_): prob of the hidden nodes after k sampling given the values of the vk
    """
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)
    
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

## Training the RBM


In [19]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(0, nb_users - batch_size, batch_size): #0-100, 100-200, ..., 800-900
    vk = training_set[id_user : id_user + batch_size]
    v0 = training_set[id_user : id_user + batch_size]
    ph0,_ = rbm.sample_h(v0)
    for k in range(10):
      _,hk = rbm.sample_h(vk) #hidden nodes obtained in the kth con div
      _,vk = rbm.sample_v(hk) #visible nodes obtained in the kth con div
      vk[v0<0] = v0[v0<0] #freeze not rated movies
    phk,_ = rbm.sample_h(vk)
    rbm.train(v0, vk, ph0, phk)
    train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
    s += 1.
  print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


epoch: 1 loss: tensor(0.3468)
epoch: 2 loss: tensor(0.2371)
epoch: 3 loss: tensor(0.2476)
epoch: 4 loss: tensor(0.2501)
epoch: 5 loss: tensor(0.2467)
epoch: 6 loss: tensor(0.2467)
epoch: 7 loss: tensor(0.2459)
epoch: 8 loss: tensor(0.2482)
epoch: 9 loss: tensor(0.2493)
epoch: 10 loss: tensor(0.2458)


## Testing the RBM


In [20]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2491)


## Predicting for one user

In [30]:
def predict(id_user, training_set, test_set):
    """
    Predicts the rating for each unrated movie in the test set for a given user.
    
    Parameters:
    id_user (int): The ID of the user for whom recommendations are being made.
    training_set (numpy array): A matrix of user ratings for all movies.
    test_set (numpy array): A matrix of user ratings for all movies, with zeros indicating unrated movies.
    
    Returns:
    numpy array: An array of predicted ratings for each unrated movie.
    """
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        probs, preds = rbm.sample_v(h)
        
        #v are the predictions, let's give back the 1 predictions, if the user did not rated that movie (has a value of -1)
        indices = np.where(preds[0] - training_set[id_user:id_user+1][0] == 2)[0]
        
        return probs, preds, indices
    else:
        return "There are not enough ratings by the user to recommend new movies"

probs, preds, indices = predict(0, training_set, test_set)

movies.iloc[indices, :]

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
9,10,GoldenEye (1995),Action|Adventure|Thriller
11,12,Dracula: Dead and Loving It (1995),Comedy|Horror
16,17,Sense and Sensibility (1995),Drama|Romance
22,23,Assassins (1995),Thriller
...,...,...,...
1672,1721,Titanic (1997),Drama|Romance
1673,1722,Tomorrow Never Dies (1997),Action|Romance|Thriller
1674,1723,Twisted (1996),Comedy|Drama
1676,1725,"Education of Little Tree, The (1997)",Drama


Probabilities

In [33]:
print(f"Probs: {np.around(np.array(probs),2)}")

Probs: [[1.   0.15 0.03 ... 0.01 0.98 0.  ]]


Predictions

In [35]:
print(f"Preds: {preds[0]}")

Preds: tensor([1., 0., 0.,  ..., 0., 1., 0.])
